In [1]:
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import mean_squared_error, make_scorer

import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import matplotlib.pyplot as plt
import seaborn as sns

import re
from tqdm import tqdm
import random

import time
import pandas as pd
from src.utils import Logger, Setting, models_load
from src.data import context_data_load, context_data_split, context_data_loader
from src.data import dl_data_load, dl_data_split, dl_data_loader
from src.data import image_data_load, image_data_split, image_data_loader
from src.data import text_data_load, text_data_split, text_data_loader
from src.train import train, test

In [2]:
class argparse: # dummy class
    def __init__(self):
        pass
    
args = argparse()

############### USER DEFINE
model = 'DCN'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
seed = 42 # 바꾸지 마시오!

############### BASIC OPTION
args.data_path='data/'
args.saved_model_path='./saved_models'
args.model=model
args.data_shuffle=True
args.test_size=0.2
args.seed=seed
args.use_best_model=True

############### TRAINING OPTION
args.batch_size=1024
args.epochs=30
args.lr=1e-3
args.loss_fn='RMSE'
args.optimizer='ADAM'
args.weight_decay=1e-6
args.early_stop=3

############### GPU
args.device=device

############### FM, FFM, NCF, WDN, DCN Common OPTION
args.embed_dim=16
args.dropout=0.2
args.mlp_dims=(16, 16)

############### DCN
args.num_layers=3

############### CNN_FM
args.cnn_embed_dim=64
args.cnn_latent_dim=12

############### DeepCoNN
args.vector_create=False
args.deepconn_embed_dim=32
args.deepconn_latent_dim=10
args.conv_1d_out_dimt=50
args.kernel_size=3
args.word_dim=768
args.out_dim=32

In [3]:
Setting.seed_everything(args.seed)

In [4]:
data = dl_data_load(args)

ids : 68092 isbn : 149570 author : 60290


In [5]:
# sample_submission = pd.read_csv(args.data_path + 'sample_submission_filtered.csv')
sample_submission = pd.read_csv(args.data_path + 'sample_submission.csv')

In [6]:
def rmse(y, y_pred):
    return mean_squared_error(y, y_pred, squared = False)

n_splits = 10
cv = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)

folds = []
for train_idx, valid_idx in cv.split(data['train'].drop('rating', axis = 1), data['train']['rating']):
    folds.append((train_idx,valid_idx))

    
def objective(trial):
    args.batch_size = trial.suggest_categorical('batch_size',[256, 512, 1024, 2048])
    args.lr = trial.suggest_loguniform('lr',0.001,0.01)
    args.weight_decay = trial.suggest_loguniform('weight_decay',1e-07,1e-06)
    args.embed_dim = trial.suggest_int('embed_dim', 1, 16)
    args.dropout = trial.suggest_categorical("dropout",[0.25,0.4,0.55, 0.7])
    args.num_layers = trial.suggest_int('num_layers',1 , 4)
    args.mlp_dims = [trial.suggest_int('mlp_dims',1,16)]*args.num_layers
    
    setting = Setting()

    log_path = setting.get_log_path(args)
    setting.make_dir(log_path)

    logger = Logger(args, log_path)
    logger.save_args()
    
    model = models_load(args,data)
    minimum_loss, model = train(args, model, data, logger, setting)
    
    return minimum_loss

for fold in range(0, n_splits):
    print(f'===================================={fold+1}============================================')
    train_idx, valid_idx = folds[fold]
    X_train = data['train'].drop('rating', axis = 1).iloc[train_idx]
    X_valid = data['train'].drop('rating', axis = 1).iloc[valid_idx]
    y_train = data['train']['rating'].iloc[train_idx]
    y_valid = data['train']['rating'].iloc[valid_idx]

    data['X_train'], data['X_valid'], data['y_train'], data['y_valid'] = X_train, X_valid, y_train, y_valid
    data = dl_data_loader(args, data)
    
    sampler = optuna.samplers.TPESampler(seed=seed)
    study = optuna.create_study(
        study_name = 'dcnc_parameter_opt',
        direction = 'minimize',
        sampler = sampler)
    
    study.optimize(objective, n_trials=10)

    print('best params {} :'.format(fold+1), study.best_value)
    print(study.best_params)
    temp = args.__dict__
    for key, value in study.best_params.items():
        if key == 'mlp_dims':
            temp[key] = [value] * study.best_params['num_layers']
        else:
            temp[key] = value
    
    setting = Setting()

    log_path = setting.get_log_path(args)
    setting.make_dir(log_path)

    logger = Logger(args, log_path)
    logger.save_args()
    
    model = models_load(args, data)
    _, model = train(args, model, data, logger, setting)
    
    predicts = test(args, model, data, setting)

    sample_submission[f'pred_{fold}'] = predicts
    print(f'================================================================================\n\n')

[I 2023-04-19 11:22:18,590] A new study created in memory with name: dcnc_parameter_opt


====================================1============================================


/tmp/ipykernel_86768/510847217.py:14: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  args.lr = trial.suggest_loguniform('lr',0.001,0.01)
/tmp/ipykernel_86768/510847217.py:15: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  args.weight_decay = trial.suggest_loguniform('weight_decay',1e-07,1e-06)
  0%|          | 0/30 [00:00<?, ?it/s]/opt/conda/lib/python3.8/site-packages/torch/nn/modules/container.py:435: UserWarning: Setting attributes on ParameterList is not supported.
  warnings.warn("Setting attributes on ParameterList is not supported.")
  3%|▎         | 1/30 [00:05<02:38,  5.47s/it]

Epoch: 1, Train_loss: 3.869, valid_loss: 2.378


  7%|▋         | 2/30 [00:10<02:29,  5.33s/it]

...

Epoch: 3, Train_loss: 2.075, valid_loss: 2.177


 13%|█▎        | 4/30 [00:16<01:45,  4.05s/it]

Epoch: 4, Train_loss: 1.879, valid_loss: 2.185


 13%|█▎        | 4/30 [00:18<01:59,  4.61s/it]
[W 2023-04-19 11:35:26,713] Trial 4 failed with parameters: {'batch_size': 256, 'lr': 0.0012260057359187524, 'weight_decay': 1.5703008378806713e-07, 'embed_dim': 1, 'dropout': 0.7, 'num_layers': 2, 'mlp_dims': 5} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/opt/conda/lib/python3.8/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
  File "/tmp/ipykernel_86768/510847217.py", line 30, in objective
    minimum_loss, model = train(args, model, data, logger, setting)
  File "/opt/ml/level1_bookratingprediction-recsys-02/src/train/trainer.py", line 50, in train
    loss.backward()
  File "/opt/conda/lib/python3.8/site-packages/torch/tensor.py", line 221, in backward
    torch.autograd.backward(self, gradient, retain_graph, create_graph)
  File "/opt/conda/lib/python3.8/site-packages/torch/autograd/__init__.py", line 130, in backward
    Variable.

KeyboardInterrupt: 

In [ ]:
sample_submission['rating'] = 0
final_pred = sample_submission['rating'] 

for fold in range(n_splits):
    final_pred += sample_submission[f'pred_{fold}']

sample_submission['rating'] = final_pred / n_splits
temp = sample_submission.copy()

sample_submission = sample_submission[['user_id', 'isbn', 'rating']]

In [ ]:
sample_submission.head()

In [ ]:
sample_submission['rating'].isna().sum()

In [ ]:
sample_submission.to_csv(args.data_path + '/submission_DCNC.csv', index=False)

In [ ]:
sns.kdeplot(sample_submission['rating'])

In [ ]:
sample_submission.shape